In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
# Define hyperparameters for the training and optimization:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 0.01
momentum = 0.5
log_interval = 10
# We will set a fixed random seed to ensure reproducibility of training.
torch.manual_seed(1)
class MLP(nn.Module):
      def __init__(self):
          super(MLP, self).__init__()
          self.flatten = nn.Flatten()
          self.fc1 = nn.Linear(32*32*3, 512)
          self.bn1 = nn.BatchNorm1d(512)
          self.relu1 = nn.ReLU()
          self.drop1 = nn.Dropout(0.5)

          self.fc2 = nn.Linear(512, 256)
          self.bn2 = nn.BatchNorm1d(256)
          self.relu2 = nn.ReLU()
          self.drop2 = nn.Dropout(0.5)

          self.fc3 = nn.Linear(256, 128)
          self.bn3 = nn.BatchNorm1d(128)
          self.relu3 = nn.ReLU()

          self.fc4 = nn.Linear(128, 64)
          self.bn4 = nn.BatchNorm1d(64)
          self.relu4 = nn.ReLU()

          self.fc5 = nn.Linear(64, 10)  # No non-linearity in the last layer
      def forward(self, x):
        x = self.flatten(x)
        x = self.drop1(self.relu1(self.bn1(self.fc1(x))))
        x = self.drop2(self.relu2(self.bn2(self.fc2(x))))
        x = self.relu3(self.bn3(self.fc3(x)))
        x = self.relu4(self.bn4(self.fc4(x)))
        x = self.fc5(x)  # Output logits
        return x


def generate_training_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

def generate_testing_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=generate_training_transforms())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=generate_testing_transforms())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

example = next(iter(trainloader))

type(example)

print("Example is of length {}".format(len(example)))
print("Type of first argument is {}".format(type(example[0])))
print("Type of second argument is {}".format(type(example[1])))

print("First argument has shape: {}".format(example[0].shape))
print("Second argument has shape {}".format(example[1].shape))
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

network = MLP()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum) # Initialize our optimizer

def train_epoch(network, trainloader, optimizer):
    network.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for data, target in trainloader:
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples += target.size(0)
        probabilities = F.softmax(output, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1)
        correct_predictions += (predicted_classes == target).sum().item()

    average_loss = total_loss / len(trainloader)
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

def test_epoch(network, testloader):
      network.eval()
      total_loss = 0
      correct_predictions = 0
      total_samples = 0
      with torch.no_grad():
        for data, target in testloader:
            output = network(data)
            loss = F.cross_entropy(output, target)
            total_loss += loss.item()
            total_samples += target.size(0)
            probabilities = F.softmax(output, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1)
            correct_predictions += (predicted_classes == target).sum().item()
            average_loss = total_loss / len(testloader)
            accuracy = correct_predictions / total_samples
            return average_loss, accuracy
            test_loss, test_accuracy = test_epoch(network, testloader)
            print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}')
for epoch in range(1, n_epochs + 1):
    train_loss, train_accuracy = train_epoch(network, trainloader, optimizer)
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}')
train_loss, train_accuracy = test_epoch(network, trainloader)
test_loss, test_accuracy = test_epoch(network, testloader)
print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}')
network = MLP()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

Files already downloaded and verified
Files already downloaded and verified
Example is of length 2
Type of first argument is <class 'torch.Tensor'>
Type of second argument is <class 'torch.Tensor'>
First argument has shape: torch.Size([64, 3, 32, 32])
Second argument has shape torch.Size([64])
Epoch 1, Train Loss: 1.9084, Train Accuracy: 0.31
Epoch 2, Train Loss: 1.7132, Train Accuracy: 0.39
Epoch 3, Train Loss: 1.6430, Train Accuracy: 0.41
Epoch 4, Train Loss: 1.5961, Train Accuracy: 0.43
Epoch 5, Train Loss: 1.5617, Train Accuracy: 0.44
Epoch 6, Train Loss: 1.5315, Train Accuracy: 0.45
Epoch 7, Train Loss: 1.5051, Train Accuracy: 0.46
Epoch 8, Train Loss: 1.4858, Train Accuracy: 0.47
Epoch 9, Train Loss: 1.4648, Train Accuracy: 0.48
Epoch 10, Train Loss: 1.4501, Train Accuracy: 0.48
Train Loss: 0.0015, Train Accuracy: 0.61
Test Loss: 0.1322, Test Accuracy: 0.53
